<a href="https://colab.research.google.com/github/lcpbrandao/Basic_ML_training/blob/master/ML_Titanic_Laislla_(Logistic_Reg).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import os

%env SPARK_HOME=/opt/spark-2.4.3

exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
sqlContext = SQLContext(sc)

env: SPARK_HOME=/opt/spark-2.4.3


FileNotFoundError: [Errno 2] No such file or directory: '/opt/spark-2.4.3\\python/pyspark/shell.py'

In [ ]:
data = spark.read.csv("titanic_train.csv", header = True, inferSchema = True )

In [ ]:
data.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [ ]:
data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [ ]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
df = data.drop("Name", "Ticket")
df.show(5)

+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
|          1|       0|     3|  male|22.0|    1|    0|   7.25| null|       S|
|          2|       1|     1|female|38.0|    1|    0|71.2833|  C85|       C|
|          3|       1|     3|female|26.0|    0|    0|  7.925| null|       S|
|          4|       1|     1|female|35.0|    1|    0|   53.1| C123|       S|
|          5|       0|     3|  male|35.0|    0|    0|   8.05| null|       S|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
only showing top 5 rows



In [ ]:
df.describe("Age").show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [ ]:
df = df.na.fill({'Age': df.approxQuantile('Age', [0.5], 0)[0]})
df = df.na.fill({'Cabin': "X"})

In [ ]:
#Outra solução:

mAge = df.selectExpr('percentile(Age, 0.5) as p').collect()
age = mAge[0].asDict()['p']

In [ ]:
#Outra solução:

mAge = df.selectExpr('percentile(Age, 0.5) as p')
mAge.show()
mAgeC = mAge.collect()
print(mAgeC)
print(mAgeC[0])
print(mAgeC[0].asDict())
age = mAgeC[0].asDict()['p']

+----+
|   p|
+----+
|28.0|
+----+

[Row(p=28.0)]
Row(p=28.0)
{'p': 28.0}


In [ ]:
df.show(5)

+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
|          1|       0|     3|  male|22.0|    1|    0|   7.25|    X|       S|
|          2|       1|     1|female|38.0|    1|    0|71.2833|  C85|       C|
|          3|       1|     3|female|26.0|    0|    0|  7.925|    X|       S|
|          4|       1|     1|female|35.0|    1|    0|   53.1| C123|       S|
|          5|       0|     3|  male|35.0|    0|    0|   8.05|    X|       S|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+
only showing top 5 rows



In [ ]:
df = df.dropna()
df = df.drop("Cabin")
df.count()

889

In [ ]:
#TRANSFORMAR COLUNA DE SEXO DE CATEGÓRICA PARA ÍNDICE

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
df = indexer.fit(df).transform(df)
df.select(df.Sex, df.SexIndex).show()

In [ ]:
#TRANSFORMAR COLUNA DE Embarked DE CATEGÓRICA PARA ÍNDICE (Index - 1a parte)

indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndex")
df = indexer.fit(df).transform(df)
df.select(df.Embarked, df.EmbarkedIndex).show()

In [ ]:
#TRANSFORMAR COLUNA DE Embarked DE CATEGÓRICA PARA ÍNDICE (OneHotEncoder - 2a parte)
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(inputCols=["EmbarkedIndex"],outputCols=["EmbarkedVec"])
model = encoder.fit(df)
df2 = model.transform(df)
df2.select(df2.EmbarkedVec).show()

+-------------+
|  EmbarkedVec|
+-------------+
|(2,[0],[1.0])|
|(2,[1],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|    (2,[],[])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[1],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|    (2,[],[])|
|(2,[0],[1.0])|
|(2,[0],[1.0])|
|(2,[1],[1.0])|
+-------------+
only showing top 20 rows



In [ ]:
df2.show()

+-----------+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-------------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|EmbarkedIndex|  EmbarkedVec|
+-----------+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-------------+
|          1|       0|     3|  male|22.0|    1|    0|   7.25|       S|     0.0|          0.0|(2,[0],[1.0])|
|          2|       1|     1|female|38.0|    1|    0|71.2833|       C|     1.0|          1.0|(2,[1],[1.0])|
|          3|       1|     3|female|26.0|    0|    0|  7.925|       S|     1.0|          0.0|(2,[0],[1.0])|
|          4|       1|     1|female|35.0|    1|    0|   53.1|       S|     1.0|          0.0|(2,[0],[1.0])|
|          5|       0|     3|  male|35.0|    0|    0|   8.05|       S|     0.0|          0.0|(2,[0],[1.0])|
|          6|       0|     3|  male|28.0|    0|    0| 8.4583|       Q|     0.0|          2.0|    (2,[],[])|
|          7|       0|     1

In [ ]:
#TRANSFORMAR FEATURES EM VETOR PARA PODER COLOCAR NO MODELO

from pyspark.ml.feature import VectorAssembler

vAssembler = VectorAssembler(inputCols = ['PassengerId',
                                                'Pclass',
                                                'Age',
                                                'SibSp',
                                                'Parch',
                                                'Fare',
                                                'SexIndex',
                                                'EmbarkedVec'
                                         ], outputCol = 'Survived')
data = vAssembler.transform(df) # Adiciona uma nova coluna a df
data = data.select(['features', 'label']) # Seleciona apenas a nova coluna e a saída

In [ ]:
#SEPARAR BASE DE TREINAMENTO E TESTE

In [ ]:
#MODELO REGRASSÃO LOGÍSTICA

In [ ]:
#EFICIÊNCIA - RESULTADOS DO TREINO